In [1]:
import torch
import torchaudio
from tqdm import tqdm
from os import listdir
from os.path import isfile, join, normpath
# import wget
import scipy.io
from pprint import pprint

## Get Audio Data From Online Drive

In [ ]:
Dateset_URL = ''
HOME = 'C:\Users\claud\OneDrive\Documents\School\T7\50.039 DL\Deep_Learning_Denoiser\noisy_samples\noise_sample_177659.wav'

dataset = wget.download(URL)

In [2]:
path = '..\\data\\clean_audio_WAV\\common_voice_en_1.wav'
waveform, sample_rate = torchaudio.load(path)
print(waveform.shape,sample_rate)

resampler = torchaudio.transforms.Resample(48000, 22050)
resampled_waveform = resampler(waveform)
print(max(resampled_waveform[0]))


torch.Size([1, 107136]) 48000
tensor(0.2032)


In [8]:
splits = torch.split(waveform, 200_000, dim=1)
len(splits)

1

In [9]:
path = '..\\data\\noisy_audio_WAV\\noisy_0.wav'
waveform, sample_rate = torchaudio.load(path)
print(waveform.shape,sample_rate)

resampler = torchaudio.transforms.Resample(22050, 48000)
resampled_waveform = resampler(waveform)
print(resampled_waveform.shape)
print(resampled_waveform.dtype)


torch.Size([1, 49216]) 22050
torch.Size([1, 107137])
torch.float32


In [4]:
# path = '.\\noisy_samples\\common_voice_en_1.mp3'
path = '..\\data\\clean_audio_tensors\\common_voice_en_1.pt'
# path = '.\\noisy_samples\\noise_sample_62094.wav'
waveform = torch.load(path)
waveform.shape

torch.Size([1, 107136])

In [5]:
path = '..\\data\\noisy_audio_tensors\\noisy_0.pt'
waveform = torch.load(path)
waveform.shape

torch.Size([1, 49216])

## Finding Silent Audio Files

In [16]:
path_input = '../data/clean_audio_tensors'
path_output = '../data/noisy_audio_tensors'

silent_audio = []

file_data = [f for f in listdir(path_input) if isfile (join(path_input, f))]
for idx, line in enumerate(tqdm(file_data)):
    if ( line[-1:] == '\n' ):
        line = line[:-1]

    # Reading Song
    songname = path_input + '/' + line
    # save_dest = path_output + '/' + line.split('.')[0] + '.pt'
    # save_dest = path_output + '/' + line.split('.')[0] + '.pt'
    # print(save_dest)
    output_audio = path_output + '/noisy_' + str(idx) + '.pt'

    waveform = torch.load(songname)[0]

    if waveform.sum().item() == 0:
        silent_audio.append((songname, output_audio))


pprint(silent_audio)

100%|██████████| 10000/10000 [00:04<00:00, 2400.96it/s]

[('./data/clean_audio_tensors/common_voice_en_100543.pt',
  './data/noisy_audio_tensors/noisy_613.pt'),
 ('./data/clean_audio_tensors/common_voice_en_100544.pt',
  './data/noisy_audio_tensors/noisy_614.pt'),
 ('./data/clean_audio_tensors/common_voice_en_100545.pt',
  './data/noisy_audio_tensors/noisy_615.pt'),
 ('./data/clean_audio_tensors/common_voice_en_101610.pt',
  './data/noisy_audio_tensors/noisy_1945.pt'),
 ('./data/clean_audio_tensors/common_voice_en_101611.pt',
  './data/noisy_audio_tensors/noisy_1946.pt'),
 ('./data/clean_audio_tensors/common_voice_en_101612.pt',
  './data/noisy_audio_tensors/noisy_1947.pt'),
 ('./data/clean_audio_tensors/common_voice_en_10327.pt',
  './data/noisy_audio_tensors/noisy_4433.pt'),
 ('./data/clean_audio_tensors/common_voice_en_10328.pt',
  './data/noisy_audio_tensors/noisy_4444.pt'),
 ('./data/clean_audio_tensors/common_voice_en_10329.pt',
  './data/noisy_audio_tensors/noisy_4455.pt'),
 ('./data/clean_audio_tensors/common_voice_en_10330.pt',
  '.

In [23]:
print(join(path_input,path_output))

./data/clean_audio_tensors\./data/noisy_audio_tensors


In [18]:
print(len(silent_audio))

44


In [ ]:
import shutil

silent_path = '../data/silent_audio'

for files in silent_audio:
    clean, noisy = files[0], files[1]
    
    clean_filename = clean.split('/')[-1]
    noisy_filename = noisy.split('/')[-1]

    shutil.move(clean, join(silent_path,clean_filename))
    shutil.move(noisy, join(silent_path,noisy_filename))

    

In [5]:
import torch.nn.functional as F
import os

path_input = '../data/clean_audio_tensors'
path_output = '../data/clean_audio_tensors_v2'

silent_audio = []

def pad(data, size):
    assert data.size(dim=1) <= size
    if data.size(dim=1) == size:
        return data
    padded_data  = F.pad(data, pad=(0, size - data.shape[1]))
    return padded_data  



file_data = [f for f in listdir(path_input) if isfile (join(path_input, f))]
for idx, line in enumerate(tqdm(file_data)):
    if ( line[-1:] == '\n' ):
        line = line[:-1]

    # Reading Song
    songname = path_input + '/' + line
    # save_dest = path_output + '/' + line.split('.')[0] + '.pt'
    # save_dest = path_output + '/' + line.split('.')[0] + '.pt'
    # print(save_dest)
    # output_audio = path_output + '/noisy_' + str(idx) + '.pt'

    waveform = torch.load(songname)

    splits = list(torch.split(waveform, 22_050, dim=1))
    splits[-1] = pad(splits[-1], 22050)
    

    for s_idx, split in enumerate(splits):
        fn = 'clean' + str(idx) + '_' + str(s_idx) + '.pt'
        torch.save(split, os.path.join(path_output, fn))

100%|██████████| 9956/9956 [02:32<00:00, 65.30it/s]


In [7]:
path_input = '../data/Hl2_Rebel-Ragdoll485-573931361.wav'

waveform, samplerate = torchaudio.load(path_input)

In [8]:
waveform.shape

torch.Size([2, 123942])

In [12]:
samplerate

44100

In [11]:
a = torch.mean(waveform, dim=0).unsqueeze(0)

a.shape

torch.Size([1, 123942])

In [13]:
path_output = '../data/torture.wav'
torchaudio.save(path_output, a, 44100)

# Testing Sanity Check Files


In [9]:
input_path = '..\\data\\clean_sanity_check\\p234_002.wav'

waveform, sr = torchaudio.load(input_path)

print(sr)

48000


In [12]:
waveform[0][0]

tensor(0.0010)